In [1]:
import torch
import numpy as np
from stable_baselines3 import DDPG, SAC, TD3, PPO
from stable_baselines3.common.noise import NormalActionNoise

import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector import HumanMoveSectorAction
import common.mlflow_sb3_helper as mlf

In [2]:
mlflow_server = mlf.MLflowServerHelper("http://192.168.0.206:2670")

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
models = {'PPO':PPO, 'DDPG':DDPG, 'SAC':SAC, 'TD3':TD3}
activations = {'ReLU':torch.nn.ReLU, 'Tanh':torch.nn.Tanh}
device = "cuda" if torch.cuda.is_available() else "cpu"

mlflow_path = {}

In [3]:
env = HumanMoveSectorAction(target_point_rand=False, object_ignore=True)
env_disc = HumanMoveSectorAction(continuous=False, target_point_rand=False, object_ignore=True)

env_render = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, render_mode='rgb_array')
env_disc_render = HumanMoveSectorAction(continuous=False, target_point_rand=False, object_ignore=True, render_mode='rgb_array')


In [4]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'ReLU',
        'pi': [256, 256],
        'qf': [256, 256],
        'vf': [256, 256],
    },
    'training': {
        'iteration_count': 1,
        'episode_count': 69000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.003,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 512,
        'tau': 0.005,
        'gamma': 0.99,
        'verbose': 0,
        'device': device,
        'DQN':{
            'fraction_eps' : 0.6,
            'start_eps' : 1.0,
            'final_eps' : 0.3,
        },
    },
    'validation': {
        'validate_agent_every_n_eps': 10000,
        'log_interval': 10
    },
    'evaluation': {
        'episode_count': 1
    }
}

policy_kwargs = dict(activation_fn = activations[exp_params['net']['activation']],
                     net_arch = dict(pi=exp_params['net']['pi'], qf=exp_params['net']['qf'], vf=exp_params['net']['vf']))



In [5]:
#!!!!! Новый эксперимент
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)

models_exclude = None

2024/09/27 16:23:02 INFO mlflow.tracking.fluent: Experiment with name 'env_MoveSector_IgnoreObst_exp_2709_162302' does not exist. Creating a new experiment.


In [4]:
#!!!!! Дорасчет старого эксперимента
exp_params['env_name'] = env.name()
experiment_id = 161
exp_name = mlflow_server.get_experiment(experiment_id)
models_exclude = {'PPO':True, 'DQN': True, 'DDPG': True, 'SAC': True, 'TD3': False}

print(exp_name)

env_HumanMoveSimple_exp_0709_144744


In [6]:
exp_params['exp_id'] = experiment_id


i = 0
for name, m in models.items():
    if models_exclude != None and models_exclude[name] == True:
        continue
    print(name)
    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    model = 0
    if name == 'TD3':
        n_actions = env.action_space.shape[-1]
        action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
        model = m(  exp_params['training']['policy'], 
                    env,
                    #policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    action_noise=action_noise,
                    device=device,
                    )
    elif name == 'DQN':
        model = m(  exp_params['training']['policy'], 
                    env_disc,
                    #policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    exploration_fraction=exp_params['training']['DQN']['fraction_eps'],
                    exploration_initial_eps= exp_params['training']['DQN']['start_eps'],
                    exploration_final_eps= exp_params['training']['DQN']['final_eps'],
                    device=device,
                    )
    elif name == 'PPO':
        model = models['PPO'](  exp_params['training']['policy'], 
                    env,
                    #policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    batch_size=exp_params['training']['batch_size'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    device=device,
                    )
    else:
        model = m(  exp_params['training']['policy'], 
                    env,
                    #policy_kwargs=policy_kwargs,
                    learning_rate=exp_params['training']['learning_rate'],
                    buffer_size=exp_params['training']['buffer_size'],
                    learning_starts=exp_params['training']['learning_starts'],
                    batch_size=exp_params['training']['batch_size'],
                    tau=exp_params['training']['tau'],
                    gamma=exp_params['training']['gamma'],
                    verbose=exp_params['training']['verbose'],
                    device=device,
                    )

    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_disc_render if name == 'DQN' else env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

    mlflow_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'

    i = i + 1


PPO


Output()

Moviepy - Building video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4
DDPG


Output()

...

Moviepy - Building video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4
TD3


Output()

Moviepy - Building video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4.
Moviepy - Writing video env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4



Moviepy - Done !
Moviepy - video ready env_MoveSector_IgnoreObst_exp_2709_162302//agent.mp4


In [7]:
mlflow_path

{'PPO': 'mlflow-artifacts:/283/5648681ab1584d8cbdbf11b8bc6bbb96/artifacts/env_MoveSector_IgnoreObst_exp_2709_143940/sb3/model.zip',
 'DQN': 'mlflow-artifacts:/283/24650359f0c84615919289a269213598/artifacts/env_MoveSector_IgnoreObst_exp_2709_143940/sb3/model.zip',
 'DDPG': 'mlflow-artifacts:/283/4cf9d6a9163a4af7a350aea3962a5146/artifacts/env_MoveSector_IgnoreObst_exp_2709_143940/sb3/model.zip',
 'SAC': 'mlflow-artifacts:/283/b8a3c320d2f241edab2a36348bb4161c/artifacts/env_MoveSector_IgnoreObst_exp_2709_143940/sb3/model.zip'}

In [9]:
experiment_id = 187

In [10]:
mlflow_path = {  'PPO': 'mlflow-artifacts:/187/185a657f2cfd4f48b07426cbdddb8ed4/artifacts/env_MoveSector_IgnoreObst_exp_1409_181804/sb3/model.zip',
 'DQN': 'mlflow-artifacts:/187/66cdac18e0cc4ba2916614fc952586e7/artifacts/env_MoveSector_IgnoreObst_exp_1409_181804/sb3/model.zip',
 'DDPG': 'mlflow-artifacts:/187/3b3cbabb5350427683f9e55edd08afda/artifacts/env_MoveSector_IgnoreObst_exp_1409_181804/sb3/model.zip',
 'SAC': 'mlflow-artifacts:/187/b468c2e2c1a347f2abf60b1a6d1e2054/artifacts/env_MoveSector_IgnoreObst_exp_1409_181804/sb3/model.zip',
 'TD3': 'mlflow-artifacts:/187/70cf3848fff348ad952bc63e3c29f756/artifacts/env_MoveSector_IgnoreObst_exp_1409_181804/sb3/model.zip'}

In [11]:

load_path = os.path.join(str(experiment_id))
os.mkdir(load_path)
load_path

'187'

In [12]:

loadel_models = {}

for model_name, path in mlflow_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    mlflow_server.load_artifact(path,loadel_models[model_name])

In [13]:
env = HumanMoveSectorAction(target_point_rand=False, object_ignore=True)
env_disc = HumanMoveSectorAction(continuous=False, target_point_rand=False, object_ignore=True)

env_render = HumanMoveSectorAction(target_point_rand=False, object_ignore=True, render_mode='rgb_array')
env_disc_render = HumanMoveSectorAction(continuous=False, target_point_rand=False, object_ignore=True, render_mode='rgb_array')
 
env.name()

'MoveSector_IgnoreObst'

In [14]:

exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = mlflow_server.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)
models_exclude = {'PPO':False, 'DQN': False, 'DDPG': False, 'SAC': False, 'TD3': False}

env_MoveSector_IgnoreObst_exp_1409_181804


In [15]:
for name, model_path in loadel_models.items():

    if models_exclude != None and models_exclude[name] == True:
        continue
    print(name)

    exp_params['algorithm_name'] = name
    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    set_env = env_disc if name == 'DQN' or name == 'PPO' else env
    model = models[name].load(model_path + 'model.zip', env=set_env, device=device)
    art_loc, exp_name, run_id = mlflow_server.learn_and_fix(
                                model=model, 
                                env= env_disc_render if name == 'DQN' or name == 'PPO' else env_render,
                                run_name=name,
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'])

PPO


Output()

KeyboardInterrupt: 

In [11]:
shutil.rmtree(os.path.join(load_path))